In [6]:
import os
import numpy as np
import torch

In [7]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights # Importing specific model
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
from torchvision.transforms.functional import to_pil_image
import matplotlib.pyplot as plt

In [18]:
plt.rcParams["savefig.bbox"] = 'tight'

In [19]:
# 이미지 세그멘테이션 및 저장 함수
def segment_and_save_image(image_path, save_path):
    # 이미지 로드
    img = read_image(image_path)
    img = img[:3, :, :]

    # 모델 초기화
    weights = FCN_ResNet50_Weights.DEFAULT
    model = fcn_resnet50(weights=weights)
    model.eval()

    # 전처리
    preprocess = weights.transforms()
    batch = preprocess(img).unsqueeze(0)

    # 예측
    prediction = model(batch)["out"]
    normalized_masks = prediction.softmax(dim=1)

    # 원본 이미지 로드
    original_image = to_pil_image(batch[0])

    # 클래스 이름과 인덱스 매핑
    class_to_idx = {cls: idx for idx, cls in enumerate(weights.meta["categories"])}

    # 오버레이 초기화
    overlay = np.zeros((original_image.height, original_image.width, 4))

    # 마스크 생성 및 적용
    for cls_idx in class_to_idx.values():
        mask = normalized_masks[0, cls_idx].detach().numpy()
        mask = mask > 0.6
        color = np.random.rand(3)
        overlay[mask] = np.append(color, 0.5)

    # 이미지 시각화 및 저장
    # plt.imshow(original_image)
    plt.imshow(overlay, interpolation='none')
    plt.axis('off')
    plt.savefig(save_path)
    plt.close()

In [ ]:
# 이미지 파일을 순회하며 함수 호출
base_image_folder = 'streetview'
output_folder = 'streetview_assignment'

for i in range(1,4):
    image_file = f'image0{i}.png'
    segment_and_save_image(os.path.join(base_image_folder, image_file),
                           os.path.join(output_folder, image_file))

In [27]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms.functional import to_tensor

def object_detection(image_path, save_path):
    # 이미지 로드 및 전처리
    img = read_image(image_path)
    img = img[:3, :, :]  # RGBA에서 RGB로 변경
    img_tensor = img.float() / 255

    # 객체 탐지 모델 초기화 및 평가 모드 설정
    detection_model = fasterrcnn_resnet50_fpn(pretrained=True)
    detection_model.eval()

    with torch.no_grad():
        predictions = detection_model([img_tensor])

    # 원본 이미지 로드
    original_image = to_pil_image(img_tensor)

    # 객체 탐지 결과 시각화
    plt.imshow(original_image)
    for prediction in predictions[0]['boxes']:
        x1, y1, x2, y2 = prediction.numpy()
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2))

    # 이미지 저장
    plt.axis('off')
    plt.savefig(save_path)
    plt.close()


In [28]:
# 이미지 파일을 순회하며 함수 호출
base_image_folder = r'C:\Users\Jasic\Jupyter\04AI2\streetview'
output_folder2 = r'streetview_assignment_od'


if not os.path.exists(output_folder2):
    os.makedirs(output_folder2)

for i in range(1,4):
    image_file = f'image0{i}.png'
    object_detection(os.path.join(base_image_folder, image_file),
                           os.path.join(output_folder2, image_file))

In [26]:
from torchvision.models.detection import maskrcnn_resnet50_fpn

def instance_segmentation(image_path, save_path):
    # 이미지 로드 및 전처리
    img = read_image(image_path)
    img = img[:3, :, :]  # RGBA에서 RGB로 변경
    img_tensor = img.float()/ 256

    # 인스턴스 분할 모델 초기화 및 평가 모드 설정
    segmentation_model = maskrcnn_resnet50_fpn(pretrained=True)
    segmentation_model.eval()

    with torch.no_grad():
        predictions = segmentation_model([img_tensor])

    # 원본 이미지 로드
    original_image = to_pil_image(img_tensor)

    # 인스턴스 분할 결과 시각화
    plt.imshow(original_image)
    for element in range(len(predictions[0]['masks'])):
        mask = predictions[0]['masks'][element, 0]
        mask = mask.mul(255).byte().cpu().numpy()
        plt.imshow(mask, alpha=0.5)

    # 이미지 저장
    plt.axis('off')
    plt.savefig(save_path)
    plt.close()


In [29]:
# 이미지 파일을 순회하며 함수 호출
base_image_folder = r'C:\Users\Jasic\Jupyter\04AI2\streetview'
output_folder3 = 'streetview_assignment_is'

if not os.path.exists(output_folder3):
    os.makedirs(output_folder3)
    
for i in range(1,4):
    image_file = f'image0{i}.png'
    instance_segmentation(os.path.join(base_image_folder, image_file),
                           os.path.join(output_folder3, image_file))

C:\Users\Jasic\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to C:\Users\Jasic/.cache\torch\hub\checkpoints\maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [00:30<00:00, 5.76MB/s]
